<a href="https://colab.research.google.com/github/emmetorior/CN7030-/blob/main/assignment_regression_prediction_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AAPL Stock Price Regression Prediction
## Using PySpark MLLib with Feature Engineering

In [ ]:
# Install required libraries
!pip install pyspark pandas numpy

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
# Importing package
from pyspark.sql.functions import ( expr, col, lag, lead, window, stddev, mean, first, last, when, isnan, count) # we don't need all of these.
from pyspark.ml.linalg import Vectors
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

import pandas as pd
import numpy as np

In [ ]:
# Begin Spark Session
spark = SparkSession.builder.appName("Assn 1 - AAPL Stock Prediction").getOrCreate()

In [ ]:
# Sample AAPL stock data - this func based on a chunk from google - it only contains the bog standard columns - no citation needed- this function will be deleted - because we'll be reading from the dataset
def create_sample_stock_data():
    # Generate a year of simulated stock data
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='B')
    np.random.seed(88)
#FAKE DATA!!
    data = {
        'Date': dates,
        'Volume': np.random.randint(1000000, 5000000, len(dates)),
        'High': 250 + np.cumsum(np.random.normal(0.1, 1, len(dates))),    # don't worry about the values, they won't be accurate
        'Low': 200 + np.cumsum(np.random.normal(-0.1, 1, len(dates))),
        'Close': 145 + np.cumsum(np.random.normal(0, 1, len(dates)))
    }

    df = pd.DataFrame(data)
    df.to_csv('AAPL.csv', index=False)  # ACHTUNG!! - delete me later
    return df

create_sample_stock_data()

,Date,Volume,High,Low,Close
0,2023-01-02,1387032,250.416084,200.907813,145.766163
1,2023-01-03,4250464,251.297519,202.723252,145.346815
2,2023-01-04,2347535,251.365699,202.235103,144.358161
3,2023-01-05,3754993,251.854266,203.402427,143.952459
4,2023-01-06,1090474,253.354787,204.334303,144.107391
...,...,...,...,...,...
255,2023-12-25,3431565,283.156880,203.339194,186.370312
256,2023-12-26,2651564,283.762568,201.529875,186.901998
257,2023-12-27,4433040,283.349917,200.540797,185.502197
258,2023-12-28,3924969,283.646833,200.845606,185.748382


In [ ]:

df = spark.read.csv('AAPL.csv', header=True, inferSchema=True)      # infer the schema - for the moment anyway.

# Add timestamps - I need to rewrite this to set the formats directly - but I wasn't sure of which additional columns we would need.
df = df.withColumn("Date", col("Date").cast("timestamp"))

In [ ]:
# Feature Engineering
#---------------------------------------
# 1. MA9 - probably will add an MA13 later as well.
window_spec = Window.orderBy("Date").rowsBetween(-8, 0)
df = df.withColumn("9_Day_MA", mean("Close").over(window_spec))

# 2. Trading Range - daily price range for the stock
df = df.withColumn("Trading_Range", col("High") - col("Low"))

# 3. Prev Close
df = df.withColumn("Prev_Close", lag("Close").over(Window.orderBy("Date")))

# 4. QQQ Prev Close
# Will combine REAL QQQ data with REAL apple data later on...
df = df.withColumn("QQQ_Prev_Close", col("Close") * 1.05)  # Simulated proxy

# 5. StdDev from EMA 25 - Should this be MA 25? might change it back
ema_window = Window.orderBy("Date").rowsBetween(-24, 0)
df = df.withColumn("25_Day_EMA", mean("Close").over(ema_window))

df = df.withColumn("EMA_Distance_StdDev",
  stddev(expr("abs(Close - `25_Day_EMA`)")).over(ema_window)
)

# You know, when you are looking for stock regression with Bollinger Bands, the price will tend to 'regress' to the average - B.B's allow you to draw a line of standard deviation to the effect that...
# that you're saying "definitely, if price crosses this line, it will snap backwards". This is just another way of doing that.... 25 EMA or 27 EMA is what I use for my own trading, and it works well.

In [ ]:
# drop nulls
df = df.na.drop()
#n = 30
#df.drop(index=df.index[:n], inplace=True)
print(df)
# Skip some days - I'm skipping
#df = df.iloc[10:]
df.show()

DataFrame[Date: timestamp, Volume: int, High: double, Low: double, Close: double, 9_Day_MA: double, Trading_Range: double, Prev_Close: double, QQQ_Prev_Close: double, 25_Day_EMA: double, EMA_Distance_StdDev: double]
+-------------------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|               Date| Volume|              High|               Low|             Close|          9_Day_MA|     Trading_Range|        Prev_Close|    QQQ_Prev_Close|        25_Day_EMA|EMA_Distance_StdDev|
+-------------------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|2023-01-03 00:00:00|4250464|251.29751895683012|202.72325240823918|145.34681487398973|145.55648869524066| 48.57426654859094| 145.7661625164916|152.61415561768922|145.556488695240

In [ ]:
# Features
#feature_columns = [     "9_Day_MA",     "Trading_Range",     "Prev_Close",      "QQQ_Prev_Close",     "EMA_Distance_StdDev" ]
feature_columns = [
    "9_Day_MA",
    "Trading_Range",
    "Prev_Close",
    "QQQ_Prev_Close",
    "EMA_Distance_StdDev"
]

# Target: Next day's closing price
#df = df.withColumn("Next_Close", lag(-1).over(Window.orderBy("Date")))
#df = df.withColumn("Next_Close", lead("Close", 1).over(Window.orderBy("Date")))

# Set up the feature vector
#assembler = VectorAssembler( inputCols=feature_columns,    outputCol="features" )
#df = assembler.transform(df)
#df = df.withColumn("Next_Close", lag(-1).over(Window.orderBy("Date")))
df = df.withColumn("Next_Close", lead("Close", 1).over(Window.orderBy("Date")))
# Assemble feature vector
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features"
)
df = assembler.transform(df)

In [ ]:

# set up the trainind and test split 80% training.
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Train the Model - I will replace this with a different model.
#lr = LinearRegression(
#    featuresCol="features",  labelCol="Next_Close", predictionCol="predicted_close"
#)
#Train Linear Regression Model
lr = LinearRegression(
    featuresCol="features",
    labelCol="Next_Close",
    predictionCol="predicted_close"
)

model = lr.fit(train_data)

In [ ]:
#Predict
predictions = model.transform(test_data)

# Evaluate Model
evaluator = RegressionEvaluator(  labelCol="Next_Close", predictionCol="predicted_close", metricName="rmse" )
predictions.printSchema()
# Drop rows with null values in 'Next_Close' before evaluation - this was causing an error
predictions = predictions.na.drop(subset=["Next_Close"])

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error a.k.a RMSE: {rmse}")

# Display the sample predictions
predictions.select("Date", "Close", "Next_Close", "predicted_close").show()

root
 |-- Date: timestamp (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- 9_Day_MA: double (nullable = true)
 |-- Trading_Range: double (nullable = true)
 |-- Prev_Close: double (nullable = true)
 |-- QQQ_Prev_Close: double (nullable = true)
 |-- 25_Day_EMA: double (nullable = true)
 |-- EMA_Distance_StdDev: double (nullable = true)
 |-- Next_Close: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- predicted_close: double (nullable = false)

Root Mean Squared Error a.k.a RMSE: 1.1578862263119978
+-------------------+------------------+------------------+------------------+
|               Date|             Close|        Next_Close|   predicted_close|
+-------------------+------------------+------------------+------------------+
|2023-01-05 00:00:00|143.95245918177932|144.10739096998083| 144.2117373493076|
|2023-01-11 00:00:00| 144.1222097215922| 